# PATH ridership over time

Cleaned + Plotted Port Authority data from https://www.panynj.gov/path/en/about/stats.html

In [ ]:
from utz import *
from utz.colors import colors_lengthen
from datetime import timedelta
from ire import export
from path_data.paths import DATA, ALL_PQT, ALL_XLSX
from path_data.utils import mo_str

Papermill parameters:

In [ ]:
force = True
default_show = 'png'
W = 1200
H = 600
end_month = None
sheet_name = 'Months'  # .xlsx sheet name
xlsx_float_precision = 10

In [ ]:
rename_stations = {
    '9thStreet': '9th Street',
    '14thStreet': '14th Street',
    '23rdStreet': '23rd Street',
    '33rdStreet': '33rd Street',
    'Pavonia/ Newport': 'Newport',
}

In [ ]:
df = concat([
    read_parquet(pqt_path)
    for pqt_path in sorted(glob.glob(f'{DATA}/2*.pqt'))
    if fullmatch(r'\d{4}\.pqt', basename(pqt_path))
]).reset_index()
stations = df.station.apply(lambda s: rename_stations.get(s, s))
df['station'] = stations
df = df[~df.station.str.contains('TOTAL')].reset_index(drop=True)
df['dt'] = df['month']
df['year'] = df.dt.dt.year
df['month_idx'] = df.dt.dt.month
df

In [ ]:
dfs = []
for pqt_path in sorted(glob.glob(f'{DATA}/2*-day-types.pqt')):
    year = fullmatch(r'(?P<y>\d{4})-day-types\.pqt', basename(pqt_path))['y']
    dfs.append(read_parquet(pqt_path).assign(year=int(year)))
day_hists = pd.concat(dfs).reset_index().set_index(['year', 'month'])
day_hists

In [ ]:
m = (
    df.merge(
        day_hists,
        how='left',
        left_on=['year', 'month_idx'],
        right_index=True,
    )
    .rename(columns={
        'saturdays': 'sats',
        'sundays': 'suns',
    })
    .drop(columns=['year', 'month_idx', 'dt', 'avg daily', 'total'])
)

for k in ['weekday', 'sat', 'sun', 'holiday']:
    tk = f'total {k}'
    ak = f'avg {k}'
    nk = f'{k}s'
    avg = m[tk] / m[nk]
    mask = (abs(avg - m[ak]) < .51) | ((m[nk] == 0) & (m[tk] == 0))
    if not all(mask):
        err(k)
        raise ValueError(m.loc[~mask, [tk, nk, ak]])
    m[ak] = avg

m['weekends'] = m.sats + m.suns
m['total weekend'] = m['total sat'] + m['total sun']
m['avg weekend'] = m['total weekend'] / m['weekends']

cols = ['month', 'station'] + [
    p+k+s
    for p, s in [('total ', ''), ('avg ', ''), ('', 's')]
    for k in ['weekday', 'weekend', 'sat', 'sun', 'holiday']
]
cols += [ c for c in m if c not in cols ]
m = m[cols]
for c in m:
    if m[c].dtype == np.int64:
        m[c] = m[c].astype('int32')
m

In [ ]:
m.dtypes

In [ ]:
# Converting `month`s to `Timestamp`s introduces timezone issues.
# Save as string, webapp parses and creates localized date in user's timezone.
out = m.assign(month=m.month.apply(mo_str))
out

In [ ]:
if force or not exists(ALL_PQT):
    out.to_parquet(ALL_PQT, index=False, engine='fastparquet')
    err(f"Wrote {relpath(ALL_PQT)}")
if force or not exists(ALL_XLSX):
    # Normalize data to match what Excel will read back
    out_normalized = pd.DataFrame({
        k: (
            round(out[k], xlsx_float_precision)
            if out[k].dtype is np.dtype('float64')
            else out[k].astype('int64')
            if out[k].dtype is np.dtype('int32')
            else out[k]
        )
        for k in out
    })
    write_xlsx = True
    if exists(ALL_XLSX):
        out_existing = pd.read_excel(ALL_XLSX, sheet_name)
        if out_normalized.equals(out_existing):
            err(f"Skipping {ALL_XLSX} write (no changes detected)")
            write_xlsx = False
    if write_xlsx:
        out_normalized.to_excel(ALL_XLSX, index=False, engine='xlsxwriter', freeze_panes=(1, 0), sheet_name=sheet_name)
        err(f"Wrote {relpath(ALL_XLSX)}")

In [ ]:
station_hist = stations.value_counts()
station_hist

In [ ]:
assert len(station_hist.value_counts()) == 1

In [ ]:
if not default_show:
    default_show = None

if end_month is None:
    end_month = (df.month.max() + pd.Timedelta('32d')).strftime('%Y-%m')
end_month

In [ ]:
mt = m.drop(columns='station').groupby('month').sum()
month_dt = to_dt(mt.index.to_series())
months = month_dt.dt
month_idxs = months.month.rename('month_idx')
years = months.year.rename('year')
mt = sxs(years, month_idxs, mt)
mt

In [ ]:
from calendar import month_abbr
pivoted = (
    mt
    .assign(month=mt.month_idx.apply(lambda m: month_abbr[m]))
    .set_index(['month_idx', 'month'])
    [['year', 'avg weekday', 'avg weekend', 'avg sat', 'avg sun']]
    .pivot(columns='year')
    .replace(nan, 0).astype(int)
    .sort_index()
    .reset_index(level=0, drop=True)
)
pivoted

In [ ]:
import plotly.express as px
from IPython.display import Image

In [ ]:
gridcolor = '#ddd'

def default_plot(fig, hoverformat=','):
    return (
        fig
        .update_layout(
            title_x=0.5,
            paper_bgcolor='white',
            plot_bgcolor='white',
            legend=dict(traceorder='reversed'),
            hovermode='x',
        ).update_xaxes(
            tickangle=-45,
            gridcolor=gridcolor,
        ).update_traces(
            hovertemplate=None,
        ).update_yaxes(
            gridcolor=gridcolor,
            hoverformat=hoverformat,
        )
    )

In [ ]:
def stations_stack(
    y, title, name=None,
    start=None, end=None,
    dtick=None, tickangle=-45,
    width=W, height=H,
    show=default_show,
):
    if isinstance(start, str):
        start = to_dt(start)
    start = start or to_dt('2012')
    start -= timedelta(days=15)
    if isinstance(end, str):
        end = to_dt(end)
    end = end or to_dt(end_month)
    end -= timedelta(days=15)
    dims = dict(width=width, height=height)
    fig = default_plot(
        px.bar(
            m.reset_index(),
            x='month', y=y, color='station',
            title=title,
            labels={
                'station': 'Station',
                y: title,
                'month': '',
            }
        )
    ).update_xaxes(
        range=[start, end],
        dtick=dtick,
    ).update_layout(**dims)
    if name:
        path = f'img/{name}.png'
        print(f'Saving {path}')
        fig.write_image(path, **dims)
    return export(fig, name, show=show)

In [ ]:
num_years = len(mt.year.unique())
num_years

In [ ]:
px_colors = px.colors.sequential.Inferno
colors = list(reversed(colors_lengthen(px_colors, num_years)))
colors

In [ ]:
month_names = [ to_dt('2022-%02d' % i).strftime('%b') for i in range(1, 13) ]
print(' '.join(month_names))

In [ ]:
def grouped_month_plot(y, title, width=W, height=H, show=default_show):
    fig = px.bar(
        mt,
        x='month_idx', y=y,
        color=mt.year.astype(str),
        color_discrete_sequence=colors,
        labels={
            'color': 'Year',
            'month_idx': '',
            y: title,
        },
        barmode='group',
    )
    dims = dict(width=width, height=height)
    fig = fig.update_layout(
        title=f'{title} (grouped by month)',
        title_x=0.5,
        xaxis=dict(
            tickmode = 'array',
            tickvals = list(range(1, 13)),
            ticktext = month_names,
        ),
        **dims,
    )
    name = f'{y}_month_grouped.png'
    path = f'img/{name}'
    fig.write_image(path, **dims)
    return export(fig, name, show=show)

# Weekdays

In [ ]:
stations_stack(
    'avg weekday',
    'Average weekday PATH ridership',
    dtick="M12",
    name='weekdays',
)

In [ ]:
stations_stack(
    'avg weekday',
    'Average weekday PATH ridership (2020-Present)',
    name='weekdays_2020:',
    dtick='M3',
    start='2020',
)

In [ ]:
grouped_month_plot('avg weekday', 'Average Weekday Rides')

# Weekends

In [ ]:
stations_stack(
    'avg weekend',
    'Average Weekend PATH ridership',
    dtick="M12",
    name='weekends',
)

In [ ]:
stations_stack(
    'avg weekend',
    'Average Weekend PATH ridership (2020-Present)',
    dtick="M3",
    name='weekends_2020:',
    start='2020',
)

In [ ]:
grouped_month_plot('avg weekend', 'Average Weekend Rides')

# Combined

## Average Daily PATH Ridership <a id="avg-daily"></a>

In [ ]:
def lines(
    df, name, xname, y_fmt,
    hoverx=True, dir='img',
    legend_lr=False,
    ax_offset=None,
    ay_offsets=None,
    h_line=None,
    xtick=None, xtickformat=None,
    ytick=None, ytickformat=None,
    show=default_show,
    w=1000, h=600,
    **kwargs,
):
    fig = px.line(
        df,
        labels={
            'variable': '',
            'value': xname,
            'month': '',
        }
    )
    idx = df.index.to_series()
    for k, ay_offset in zip(df, ay_offsets):
        x = idx.iloc[-1]
        y = df[k].iloc[-1]
        x_str = x.strftime("%b '%y")
        y_str = format(y, y_fmt)
        fig.add_annotation(
            x=x,
            ax=idx.iloc[-ax_offset],
            axref="x",
            y=y, ayref="y", ay=y + ay_offset,
            text=f'{x_str}: {y_str}',
        )
    if h_line is not None:
        fig.add_hline(y=h_line, line=dict(color='#777', width=1)),
    fig = plots.save(
        fig,
        name=name,
        x=dict(dtick=xtick, tickformat=xtickformat),
        y=dict(dtick=ytick, tickformat=ytickformat),
        legend=(
            dict(
                yanchor="bottom",
                y=0.03,
                xanchor="right",
                x=0.99,
            ) if legend_lr else dict(
                yanchor="top",
                y=0.99,
                xanchor="right",
                x=0.99,
            )
        ),
        hoverx=hoverx,
        dir=dir,
        w=w, h=h,
        **kwargs,
    )
    return export(fig, name, show=show)

In [ ]:
lines(
    mt[['avg weekday', 'avg weekend',]].rename(columns={
        'avg weekday': 'Avg Weekday',
        'avg weekend': 'Avg Weekend',
    }),
    name='avg_day_types',
    xname = 'Daily Rides',
    xtick = "M12",
    hovertemplate='%{y:,.0f}',
    y_fmt = ',.0f',
    ax_offset=13, ay_offsets=[50_000, -50_000],
    legend_lr=False,
)

## 2020-2022 Ridership vs. 2019

In [ ]:
mt20 = mt[month_dt >= to_dt('2020')]
mt19 = mt[mt.year == 2019]
mt19s = pd.concat([ mt19 for i in range((len(mt20) + len(mt19) - 1) // len(mt19)) ]).iloc[:len(mt20)]

keys = ['avg weekday', 'avg weekend']
cmp19 = (
    sxs(
        mt19s.reset_index(drop=True)[keys].rename(columns={ key: f'{key} 2019' for key in keys }),
        mt20.reset_index()[keys + ['month']],
    )
    .set_index('month')
)
for k in keys:
    cmp19[f'{k} frac'] = cmp19[k] / cmp19[f'{k} 2019']
cmp19

## PATH Ridership: 2020-2023 (YTD) vs. 2019 <a id="vs-2019"></a>

In [ ]:
lines(
    cmp19[[f'{k} frac' for k in keys]].rename(columns={
        'avg weekday frac': 'Avg Weekday (% of 2019)',
        'avg weekend frac': 'Avg Weekend (% of 2019)',
    }),
    name='vs_2019',
    xname='% of 2019 ridership',
    xtick="M3",
    ytickformat=".0%",
    ax_offset=5, ay_offsets=[-.15, .15],
    h_line=1,
    hovertemplate='%{y:.1%}',
    y_fmt='.1%',
    legend_lr=True,
)

# {Month,Station} data

In [ ]:
export(df.set_index(['month', 'station']), 'path', fmts={'month': "%b '%y"}, per_page=20);